In [1]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from webdriver_manager.chrome import ChromeDriverManager
import time
import sqlite3
import datetime
import os

# === 設定出發與回程資訊 ===
origin = "TPE"
destination = "TYO"
departure_date = "2025-07-29"
return_date = "2025-08-05"

# === 產生資料庫完整路徑，方便確認 ===
db_path = os.path.abspath(r"C:\Users\user\Desktop\flight\db\sqlite.db")
print("📍 資料庫將儲存在：", db_path)

# === 啟動 Chrome（非 headless，人工驗證用） ===
options = webdriver.ChromeOptions()
options.add_argument('--disable-gpu')
options.add_argument('--no-sandbox')
options.add_argument('--start-maximized')
options.add_argument("user-agent=Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/115 Safari/537.36")

driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()), options=options)

# === Trip.com 查詢網址 ===
url = f"https://tw.trip.com/flights/showfarefirst?dcity={origin}&acity={destination}&ddate={departure_date}&rdate={return_date}&triptype=rt&class=y&lowpricesource=searchform&quantity=1&searchboxarg=t&nonstoponly=off&locale=zh-TW&curr=TWD"
driver.get(url)

# === 等你手動完成滑動驗證 ===
print("\n⏳ 請在開啟的瀏覽器中手動滑動驗證，完成後按 Enter 繼續...")
input("✅ 完成驗證後按 Enter >> ")

# 等待票價載入（保險起見）
time.sleep(3)

# === 擷取票價與航空公司 ===
try:
    price_element = driver.find_element(By.CSS_SELECTOR, "span[data-price]")
    price = int(price_element.get_attribute("data-price"))

    airline_element = driver.find_element(By.CSS_SELECTOR, "div.flights-name")
    airline = airline_element.text.strip()

    print("✅ 最低票價：", price)
    print("🛫 航空公司：", airline)

except Exception as e:
    print("❌ 找不到票價或航空公司：", e)
    driver.quit()
    exit()

driver.quit()

# === 寫入 SQLite 資料庫 ===
conn = sqlite3.connect(db_path)
cursor = conn.cursor()

# ✅ 第一次才建立表格，若表格已存在就不動它（不刪資料）
cursor.execute("""
CREATE TABLE IF NOT EXISTS flight_prices (
    id INTEGER PRIMARY KEY AUTOINCREMENT,
    origin TEXT,
    destination TEXT,
    departure_date TEXT,
    return_date TEXT,
    airline TEXT,
    price INTEGER,
    crawl_time TIMESTAMP
)
""")

# ✅ 每次執行都會寫入一筆新資料
cursor.execute("""
INSERT INTO flight_prices (origin, destination, departure_date, return_date, airline, price, crawl_time)
VALUES (?, ?, ?, ?, ?, ?, ?)
""", (origin, destination, departure_date, return_date, airline, price, datetime.datetime.now()))

conn.commit()
conn.close()

print("📦 已成功新增一筆資料進 SQLite 資料庫！")


📍 資料庫將儲存在： C:\Users\user\Desktop\flight\db\sqlite.db

⏳ 請在開啟的瀏覽器中手動滑動驗證，完成後按 Enter 繼續...
✅ 最低票價： 5909
🛫 航空公司： 臺灣虎航
📦 已成功新增一筆資料進 SQLite 資料庫！


C:\Users\user\AppData\Local\Temp\ipykernel_18156\3875397367.py:77: DeprecationWarning: The default datetime adapter is deprecated as of Python 3.12; see the sqlite3 documentation for suggested replacement recipes
  cursor.execute("""


# auto

In [ ]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from webdriver_manager.chrome import ChromeDriverManager
import time
import sqlite3
import datetime
import os

# === 固定查詢參數 ===
origin = "TPE"
destination = "TYO"
departure_date = "2025-07-29"
return_date = "2025-08-05"

# === 資料庫路徑（請確認與網站一致） ===
db_path = os.path.abspath(r"C:\Users\user\Desktop\flight\db\sqlite.db")
print("📍 資料庫儲存位置：", db_path)

# === 啟動瀏覽器設定（你仍需要手動處理滑動驗證） ===
options = webdriver.ChromeOptions()
options.add_argument('--disable-gpu')
options.add_argument('--no-sandbox')
options.add_argument('--start-maximized')
options.add_argument("user-agent=Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/115 Safari/537.36")

driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()), options=options)

url = f"https://tw.trip.com/flights/showfarefirst?dcity={origin}&acity={destination}&ddate={departure_date}&rdate={return_date}&triptype=rt&class=y&lowpricesource=searchform&quantity=1&searchboxarg=t&nonstoponly=off&locale=zh-TW&curr=TWD"
driver.get(url)

print("\n🔐 請手動完成滑動驗證後再按 Enter 繼續...")
input("✅ 驗證完成後請按 Enter：")

time.sleep(5)

# === 擷取資料 ===
try:
    price_element = driver.find_element(By.CSS_SELECTOR, "span[data-price]")
    price = int(price_element.get_attribute("data-price"))

    airline_element = driver.find_element(By.CSS_SELECTOR, "div.flights-name")
    airline = airline_element.text.strip()

    print("✅ 最低票價：", price)
    print("🛫 航空公司：", airline)

except Exception as e:
    print("❌ 無法取得票價或航空公司：", e)
    driver.quit()
    exit()

driver.quit()

# === 寫入資料庫 ===
conn = sqlite3.connect(db_path)
cursor = conn.cursor()

# 表格不存在就建立
cursor.execute("""
CREATE TABLE IF NOT EXISTS flight_prices (
    id INTEGER PRIMARY KEY AUTOINCREMENT,
    origin TEXT,
    destination TEXT,
    departure_date TEXT,
    return_date TEXT,
    airline TEXT,
    price INTEGER,
    crawl_time TIMESTAMP
)
""")

# 插入資料（每筆都保留）
cursor.execute("""
INSERT INTO flight_prices (origin, destination, departure_date, return_date, airline, price, crawl_time)
VALUES (?, ?, ?, ?, ?, ?, ?)
""", (
    origin,
    destination,
    departure_date,
    return_date,
    airline,
    price,
    datetime.datetime.now()
))

conn.commit()
conn.close()

print("📦 成功寫入 flight_prices！")


📍 資料庫儲存位置： C:\Users\user\Desktop\flight\db\sqlite.db

🔐 請手動完成滑動驗證後再按 Enter 繼續...
✅ 最低票價： 8772
🛫 航空公司： 韓國德威航空


OperationalError: table flights_table has no column named origin